# Data Drift training notebook

## Steps
- load curated dataset
- train data drift model (use MLFlow for experiment tracking and Minio as object storage)
- load a model from MLFlow and do the prediction

## TODO
- build model into KNative Serving
- deploy broker, trigger, serving

In [1]:
!pip install alibi-detect mlflow boto3 -q

You should consider upgrading via the '/home/barteus/Work/DSV/kubeflow-examples/data-drift/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
#Local development only

import os

os.environ['MLFLOW_TRACKING_URI'] = "http://10.1.100.48:5000"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://10.1.100.8:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"

## Load data

In [3]:
import pandas as pd
import numpy as np

In [4]:
red_wine_dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
white_wine_dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

In [5]:
df_red = pd.read_csv(red_wine_dataset_url, sep=";")
df_red.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
df_white = pd.read_csv(white_wine_dataset_url, sep=";")
df_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
red_arr = np.asarray(df_red.drop(columns=['quality']), np.float32)
white_arr = np.asarray(df_white.drop(columns=['quality']), np.float32)

## Train model

In [8]:
import mlflow

class DataDriftModel(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model

    def predict(self, context, model_input):
        return self.model.predict(model_input)

In [9]:
from sklearn.preprocessing import StandardScaler
from alibi_detect.cd import KSDrift

with mlflow.start_run() as run:
    scaler = StandardScaler()
    scaler.fit(red_arr)

    cd = KSDrift(
        red_arr,  p_val=.05, preprocess_fn=scaler.transform
    )

    result = mlflow.pyfunc.log_model(
        artifact_path='datadrift',
        python_model=DataDriftModel(cd)
    )
    model_s3_uri = f"{mlflow.get_artifact_uri()}/{result.artifact_path}"

print(result)
print()
print(model_s3_uri)

2022-04-15 12:06:13.623197: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-15 12:06:13.623235: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ModelInfo(artifact_path='datadrift', flavors={'python_function': {'cloudpickle_version': '2.0.0', 'python_model': 'python_model.pkl', 'loader_module': 'mlflow.pyfunc.model', 'python_version': '3.9.7', 'env': 'conda.yaml'}}, model_uri='runs:/519b2ab265464b7c8ee2d15d50b9e035/datadrift', model_uuid='290adf4edd4f4f5886b3dfa335b346ae', run_id='519b2ab265464b7c8ee2d15d50b9e035', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-04-15 10:06:16.268093')

s3://mlflow/0/519b2ab265464b7c8ee2d15d50b9e035/artifacts/datadrift


In [10]:
cd.predict(red_arr)

{'data': {'is_drift': 0,
  'distance': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
  'p_val': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32),
  'threshold': 0.004545454545454546},
 'meta': {'name': 'KSDrift',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.9.1'}}

In [11]:
cd.predict(white_arr)

{'data': {'is_drift': 1,
  'distance': array([0.434819  , 0.6614576 , 0.31746453, 0.49459264, 0.826595  ,
         0.5410087 , 0.7778058 , 0.48921987, 0.3593833 , 0.5257545 ,
         0.09374172], dtype=float32),
  'p_val': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 1.1482314e-09], dtype=float32),
  'threshold': 0.004545454545454546},
 'meta': {'name': 'KSDrift',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.9.1'}}

## Load and predict

Compare prediction results.

In [12]:
import mlflow
loaded_model = mlflow.pyfunc.load_model('s3://mlflow/0/f8ef7180c6f64ff7b7cbae0f2497d27d/artifacts/datadrift')
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: datadrift
  flavor: mlflow.pyfunc.model
  run_id: f8ef7180c6f64ff7b7cbae0f2497d27d

In [13]:
loaded_model.predict(red_arr)

{'data': {'is_drift': 0,
  'distance': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
  'p_val': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32),
  'threshold': 0.004545454545454546},
 'meta': {'name': 'KSDrift',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.9.1'}}

In [14]:
loaded_model.predict(white_arr)

{'data': {'is_drift': 1,
  'distance': array([0.434819  , 0.6614576 , 0.31746453, 0.49459264, 0.826595  ,
         0.5410087 , 0.7778058 , 0.48921987, 0.3593833 , 0.5257545 ,
         0.09374172], dtype=float32),
  'p_val': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 1.1482314e-09], dtype=float32),
  'threshold': 0.004545454545454546},
 'meta': {'name': 'KSDrift',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.9.1'}}

## Deployment

Build image (Optional):
- adjust the private docker registry
- download the artefact from S3/Minio
- build and push docker image
- adjust `online-data-drift.yaml` with new docker image

Deployment:
- apply `online-data-drift.yaml` - broker, service, trigger
- Get the broker URL (namespace is a part of URL) -> adjust the e2e pipeline deployment yaml with it.

In [25]:
!aws --endpoint-url $MLFLOW_S3_ENDPOINT_URL s3 cp "s3://mlflow/0/f8ef7180c6f64ff7b7cbae0f2497d27d/artifacts/datadrift" ./online-data-drift/model/ --recursive

download: s3://mlflow/0/f8ef7180c6f64ff7b7cbae0f2497d27d/artifacts/datadrift/MLmodel to online-data-drift/model/MLmodel
download: s3://mlflow/0/f8ef7180c6f64ff7b7cbae0f2497d27d/artifacts/datadrift/python_model.pkl to online-data-drift/model/python_model.pkl
download: s3://mlflow/0/f8ef7180c6f64ff7b7cbae0f2497d27d/artifacts/datadrift/conda.yaml to online-data-drift/model/conda.yaml
download: s3://mlflow/0/f8ef7180c6f64ff7b7cbae0f2497d27d/artifacts/datadrift/requirements.txt to online-data-drift/model/requirements.txt


In [26]:
!sh ./online-data-drift/build_image.sh

[sudo] password for barteus: 


In [ ]:
# namespace change - changes broker URL!!!
!kubectl apply -f online-data-drift.yaml